# Knowledge Graph RAG Query Engine


## Graph RAG

Graph RAG is an Knowledge-enabled RAG approach to retrieve information from Knowledge Graph on given task. Typically, this is to build context based on entities' SubGraph related to the task.

## GraphStore backed RAG vs VectorStore RAG

As we compared how Graph RAG helps in some use cases in [this tutorial](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html#id1), it's shown Knowledge Graph as the unique format of information could mitigate several issues caused by the nature of the "split and embedding" RAG approach.

## Why Knowledge Graph RAG Query Engine

In Llama Index, there are two scenarios we could apply Graph RAG:

- Build Knowledge Graph from documents with Llama Index, with LLM or even [local models](https://colab.research.google.com/drive/1G6pcR0pXvSkdMQlAK_P-IrYgo-_staxd?usp=sharing), to do this, we should go for `KnowledgeGraphIndex`.
- Leveraging existing Knowledge Graph, in this case, we should use `KnowledgeGraphRAGQueryEngine`.

> Note, the third query engine that's related to KG in Llama Index is `NL2GraphQuery` or `Text2Cypher`, for either exiting KG or not, it could be done with `KnowledgeGraphQueryEngine`.

Before we start the `KnowledgeGraphRAGQueryEngine` demo, let's first get ready for basic preparation of Llama Index.

In [ ]:
# For OpenAI

import os

os.environ["OPENAI_API_KEY"] = "INSERT OPENAI KEY"

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output

from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from llama_index.llms import OpenAI

from IPython.display import Markdown, display


# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm = OpenAI(temperature=0, model="text-davinci-002")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size_limit=512)

In [ ]:
# For Azure OpenAI
import os
import json
import openai
from llama_index.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index import LangchainEmbedding
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
)

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from llama_index.llms import LangChainLLM

import logging
import sys

from IPython.display import Markdown, display

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

openai.api_type = "azure"
openai.api_base = "INSERT AZURE API BASE"
openai.api_version = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "INSERT OPENAI KEY"
openai.api_key = os.getenv("OPENAI_API_KEY")

lc_llm = AzureOpenAI(
    engine="INSERT DEPLOYMENT NAME",
    temperature=0,
    model="gpt-35-turbo",
)
llm = LangChainLLM(lc_llm)

# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="nebula_docs_2_embedding",
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_llm,
)

## Prepare for NebulaGraph

We take [NebulaGraphStore](https://gpt-index.readthedocs.io/en/stable/examples/index_structs/knowledge_graph/NebulaGraphKGIndexDemo.html) as an example in this demo, thus before next step to perform Graph RAG on existing KG, let's ensure we have a running NebulaGraph with defined data schema.

This step installs the clients of NebulaGraph, and prepare contexts that defines a [NebulaGraph Graph Space](https://docs.nebula-graph.io/3.6.0/1.introduction/2.data-model/).

In [ ]:
# Create a NebulaGraph (version 3.5.0 or newer) cluster with:
# Option 0 for machines with Docker: `curl -fsSL nebula-up.siwei.io/install.sh | bash`
# Option 1 for Desktop: NebulaGraph Docker Extension https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext

# If not, create it with the following commands from NebulaGraph's console:
# CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
# :sleep 10;
# USE llamaindex;
# CREATE TAG entity(name string);
# CREATE EDGE relationship(relationship string);
# :sleep 10;
# CREATE TAG INDEX entity_index ON entity(name(256));

%pip install ipython-ngql nebula3-python

os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "<password>"  # default is "nebula"
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally

space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

Then we could instiatate a `NebulaGraphStore`, in order to create a `StorageContext`'s `graph_store` as it.

In [2]:
graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

Here, we assumed to have the same Knowledge Graph from [this turtorial](https://gpt-index.readthedocs.io/en/latest/examples/query_engine/knowledge_graph_query_engine.html#optional-build-the-knowledge-graph-with-llamaindex)

## Perform Graph RAG Query

Finally, let's demo how to do Graph RAG towards an existing Knowledge Graph.

All we need to do is to use `KnowledgeGraphRAGQueryEngine`, which is just a query engine implemented with `RetrieverQueryEngine`, and the retriver of it is `KnowledgeGraphRAGRetriever`.

Underlying, `KnowledgeGraphRAGRetriever` just do the follow steps:

- Search related Entities of the quesion/task
- Get SubGraph of those Entities (default 2-depth) from the KG
- Build Context based on the SubGraph

Please note, the way to Search related Entities could be either Keyword extraction based or Embedding based, which is controlled by argument `retriever_mode` of the `KnowledgeGraphRAGRetriever`, and supported options are:
- "keyword"
- "embedding"(not yet implemented)
- "keyword_embedding"(not yet implemented)

Here is the example on how to use `KnowledgeGraphRAGQueryEngine` and `KnowledgeGraphRAGRetriever`:

In [3]:
from llama_index.query_engine import (
    KnowledgeGraphRAGRetriever,
    KnowledgeGraphRAGQueryEngine,
)

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

query_engine = KnowledgeGraphRAGQueryEngine(retriever=graph_rag_retriever)

Then we can query it like:

In [4]:
response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

Entities processed: ['Quill', 'Peter Quill', 'Peter']
Entities processed: ['quill', 'peter', 'Peter', 'Quill', 'peter quill', 'Peter Quill']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of `subject predicate, object, predicate_next_hop, object_next_hop ...` extracted based on key entities as subject:
Peter Quill, indicated willingness to continue to play, Peter Quill, expected to return for, sequel
Peter Quill, is leader of, Guardians of the Galaxy, released in, 2014
Peter Quill, is leader of, Guardians of the Galaxy, wrote, Guardians of the Galaxy
Peter Quill, portrays, Peter Quill
Peter Quill, is leader of, Guardians of the Galaxy
Peter Quill, indicated willingness to continue to play, Peter Quill
Peter Quill, is leader of, Guardians of the Galaxy, sequel to, Guardians of the Galaxy
Peter Quill, is leader of, Guardians of the Galaxy, directed, Guardians of the Galaxy


<b>Peter Quill is the leader of the Guardians of the Galaxy. He was portrayed by an actor named Peter Quill. He indicated his willingness to continue playing the character and is expected to return for the sequel. Peter Quill also wrote the Guardians of the Galaxy. Additionally, he is associated with the release of the Guardians of the Galaxy movie in 2014.</b>

## Include nl2graphquery as Context in Graph RAG

The nature of (Sub)Graph RAG and nl2graphquery are different. No one is better than the other but just when one fits more in certain type of questions. To understand more on how they differ from the other, see [this demo](https://www.siwei.io/en/demos/graph-rag/) comparing the two.

<video width="938" height="800" 
       src="https://github.com/siwei-io/talks/assets/1651790/05d01e53-d819-4f43-9bf1-75549f7f2be9"  
       controls>
</video>

While in real world cases, we may not always know which approach works better, thus, one way to best leverage KG in RAG are fetching both retrieval results as context and letting LLM + Prompt generate answer with them all being involved.

So, optionally, we could choose to synthesise answer from two piece of retrieved context from KG:
- Graph RAG, the default retrieval method, which extracts subgraph that's related to the key entities in the question.
- NL2GraphQuery, generate Knowledge Graph Query based on query and the Schema of the Knowledge Graph, which is by default switched off.

We could set `with_nl2graphquery=True` to enable it like:

In [5]:
graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = KnowledgeGraphRAGQueryEngine(
    retriever=graph_rag_retriever_with_nl2graphquery
)

In [6]:
response = query_engine_with_nl2graphquery.query(
    "What do you know about Peter Quill?",
)
display(Markdown(f"<b>{response}</b>"))

Graph Store Query:
MATCH (p:`entity`)-[r:`relationship`]->(q:`entity`)
WHERE p.`entity`.`name` == 'Peter Quill'
RETURN q.`entity`.`name`
Graph Store Response:
{'q.entity.name': ['Guardians of the Galaxy']}
Entities processed: ['Quill', 'Peter Quill', 'Peter']
Entities processed: ['quill', 'peter', 'Peter', 'Quill', 'peter quill', 'Peter Quill']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of `subject predicate, object, predicate_next_hop, object_next_hop ...` extracted based on key entities as subject:
Peter Quill, indicated willingness to continue to play, Peter Quill, expected to return for, sequel
Peter Quill, is leader of, Guardians of the Galaxy, released in, 2014
Peter Quill, is leader of, Guardians of the Galaxy, wrote, Guardians of the Galaxy
Peter Quill, portrays, Peter Quill
Peter Quill, is leader of, Guardians of the Galaxy
Peter Quill, indicated willingness to continue to play, Peter Quill
Peter Quill, is leader of, Guardians of the Gal

<b>Based on the context information provided, we know that Peter Quill is the leader of the Guardians of the Galaxy. He is also portrayed by an actor named Peter Quill. Additionally, there is a sequel to the Guardians of the Galaxy movie that Peter Quill is expected to return for.</b>

And let's check the response's metadata to know more details of the retrival of Graph RAG with nl2graphquery by inspecting `response.metadata`.

- **text2Cypher**, it generates a Cypher Query towards the answer as the context.

```cypher
Graph Store Query: MATCH (e:`entity`)-[r:`relationship`]->(e2:`entity`)
WHERE e.`entity`.`name` == 'Peter Quill'
RETURN e2.`entity`.`name`
```
- **SubGraph RAG**, it get the SubGraph of 'Peter Quill' to build the context.

- Finally, it combined the two nodes of context, to synthesize the answer.

In [8]:
import pprint

pp = pprint.PrettyPrinter()
pp.pprint(response.metadata)

{'4131b845-559d-4eab-962f-dac1eadf84b4': {'graph_schema': 'Node properties: '
                                                          "[{'tag': 'entity', "
                                                          "'properties': "
                                                          "[('name', "
                                                          "'string')]}]\n"
                                                          'Edge properties: '
                                                          "[{'edge': "
                                                          "'relationship', "
                                                          "'properties': "
                                                          "[('relationship', "
                                                          "'string')]}]\n"
                                                          'Relationships: '
                                                          "['(:entity)-[:relationship]->(: